In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
# Apply the default theme
sns.set_theme()

In [ ]:
def parse_line(log_data):
    loss_dict = {}
    for item in log_data:
        if len(item.split("=")) == 2:
            key, val = item.split("=")
            val = float(val)
            loss_dict[key] = val
    return loss_dict

def parse_valid_line(log_data):
    loss_dict = {}
    for item in log_data[1:]:
        if len(item.split(" ")) == 2:
            key, val = item.split(" ")
            val = float(val)
            loss_dict[key] = val
    return loss_dict
    

def plot_learning_curves(exp_log_file, keys):
#     exp_log_file="../experiments/exp-%d/checkpoints%d/log" %(exp_id, check_id)
    
    train_log = {}
    dev_log = {}
    with open(exp_log_file) as f:
        for line in f:
            if len(line.split(" | ")) >= 2:
                if line.split(" | ")[2] == "train_inner":
                    log_data = line.split("|")[3].split(", ")
                    loss_dict = parse_line(log_data)
                    loss_dict["loss"] = float(log_data[1].split("=")[1])
                    iteration = loss_dict["num_updates"]
                    train_log[iteration] = loss_dict
                elif line.split(" | ")[2] == "valid":
                    log_data = line.strip().split(" | ")
                    loss_valid_dict = parse_valid_line(log_data)
                    num_updates  = loss_valid_dict["num_updates"]
                    dev_log[num_updates] = loss_valid_dict
                else:
                    continue
#     return train_log, dev_log
    for key in keys:
        x_train = list(train_log.keys())[5:]
        y_train = [train_log[i][key] for i in x_train]
        sns.lineplot(x=x_train, y=y_train)
        
        if key in dev_log[num_updates]:

            x_dev = list(dev_log.keys())[1:]
            y_dev = [dev_log[i][key] for i in x_dev]
            sns.lineplot(x=x_dev, y=y_dev)

            labels = ['train_' + key, 'dev_' + key]

            if key == "ppl":
                y_dev = [dev_log[i]["best_ppl"] for i in x_dev]
                sns.lineplot(x=x_dev, y=y_dev)
                labels.append('dev_best_ppl')

            min_dev_index = np.argmin(y_dev)
            print("Minimum dev %s value: %.3f at iter %d" %(key, y_dev[min_dev_index], x_dev[min_dev_index]))
            plt.axvline(x_dev[min_dev_index], 0, np.max(y_dev))
            plt.title(key)
            plt.legend(labels=labels)
            plt.show()


In [ ]:
keys = ["nll_loss", "ppl", "loss", "word_ins_ml", "word_reposition", "mask_ins", "lr"]
exp_id=1
all_logs = {}
for ckpd in range(1, 2):
    exp_log_file="../experiments/exp-%d/checkpoints%d/log" %(exp_id, ckpd)
    all_logs[ckpd] = plot_learning_curves(exp_log_file, keys=keys)